# 5. 파일 저장

## 이미지 저장

In [4]:
import cv2
img = cv2.imread("img.jpg", cv2.IMREAD_GRAYSCALE) # 흑백으로 이미지 불러오기
cv2.imwrite("img_save.png", img)

True

## 동영상 저장

In [6]:
import cv2
cap = cv2.VideoCapture('video.mp4')

# 코덱 정의
fourcc = cv2.VideoWriter_fourcc(*"DIVX")

width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # * 2, 2배 속도로 저장

out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))
# 저장 파일명, 코덱, fps, 크기, (width, height)

while cap.isOpened(): # 동영상 파일이 올바로 열렸는지?
    ret, frame = cap.read() # ret : 성공 여부, frame : 받아온 이미지 (프레임)
    if not ret:
        break

    out.write(frame)
    cv2.imshow("video", frame) # 영상 데이터만 저장 소리 X
    if cv2.waitKey(1) == ord("q"): # 영상 속도
        break

out.release() # 자원 해제
cap.release()
cv2.destroyAllWindows()
for i in range (1,5): # m1 창 안 닫힐때
    cv2.waitKey(1)

In [2]:
import cv2
import numpy as np

# 이미지 불러오기
img = cv2.imread('img.jpg')

# 변환 graky
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 임계값 조절
mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

# mask
mask = 255 - mask

# morphology 적용
# borderconstant 사용
kernel = np.ones((3,3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# anti-alias the mask
# blur alpha channel
mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

# linear stretch so that 127.5 goes to 0, but 255 stays 255
mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

# put mask into alpha channel
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result[:, :, 3] = mask

# 저장
cv2.imwrite('translated.png', result)

True

In [7]:
import time
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from rembg.bg import remove as remove_bg
def show_rembg(path):
    fig = plt.figure(figsize=(10, 10))

    # show original image
    fig.add_subplot(1, 2, 1)
    orig_img = Image.open(path)
    plt.imshow(orig_img)

    # show bg removed image
    fig.add_subplot(1, 2, 2)
    f = np.fromfile(path)

    started = time.time()
    result = remove_bg(f)
    elapsed = time.time() - started
    print(f'it takes {elapsed} seconds for removing bg.')

    img = Image.open(io.BytesIO(result)).convert("RGBA")
    plt.imshow(img)

# Usage
show_rembg('img.jpg')

SystemError: initialization of _internal failed without raising an exception